# Data Wrangling: Gathering data
Notes on methods for gathering data

In [1]:
import pandas as pd

We are going to look at **Rotten Tomatoes top 100 movies**

In [2]:
# Import the Rotten Tomatoes bestofrt TSV file into a DataFrame
df = pd.read_csv('bestofrt.tsv', sep='\t')

In [3]:
# Check to see if the file was imported correctly
df.head()

,ranking,critic_score,title,number_of_critic_ratings
0,1,99,The Wizard of Oz (1939),110
1,2,100,Citizen Kane (1941),75
2,3,100,The Third Man (1949),77
3,4,99,Get Out (2017),282
4,5,97,Mad Max: Fury Road (2015),370


Looking at the movies on the list we can see that the critics scores and the audience scores don't line up. Some films, like ET, have quite a big difference in scores between critics and the audience. 

We can get the data on critics and audience scores using web scraping. We will use **Beautiful soup** to parse the HTML file on the top 100 movies.

## Accessing HTML file

Code for programmatically downloading HTML file to your computer:

The two main ways to work with HTML files are:

1. Saving the HTML file to your computer (using the Requests library for example) library and reading that file into a BeautifulSoup constructor
2. Reading the HTML response content directly into a BeautifulSoup constructor (again using the Requests library for example)

In [4]:
# First method of working with HTML
#This is just the code to download 1 file. We have 100 HTML files to access so we would have to put the code in a loop 

import requests
url = "https://www.rottentomatoes.com/m/t_the_extraterrestrial"
response = requests.get(url)

# #Save HTML to file
# with open("et_the_extraterrestrial.html, mode='wb) as file:
#      file.write(response.content)

In [5]:
# Second method:
# Work with HTML in memory
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

## Working with Beautiful soup

The first thing you need to do is **make the soup**. That means passing the path to yout HTML file into a file handle, then passing that file handle into the Beautiful Soup constructor like so:

In [6]:
with open('rt_html/et_the_extraterrestrial.html') as file:
    soup = BeautifulSoup(file, "html.parser")

We can use  methods in the Beautiful soup library to easily find and extract data from this HTML. One of the most popular methods is the **find** method. Let's find the title of our movie using this method:

In [7]:
#We'll find that theres only one " <title> tag in this whle HTML document and inside this tag is the title of the movie. 
soup.find('title')

<title>E.T. The Extra-Terrestrial (1982) - Rotten Tomatoes</title>

This is the title of the webpage and not the movie title only. To get just the movie title we'll have to do some string slicing. To access the contents of these tags we can use **.contents** which returns a list of the tags children. 

In [8]:
soup.find('title').contents

['E.T. The Extra-Terrestrial\xa0(1982) - Rotten Tomatoes']

Because there's only one thing within this tag the list is one item long and we can therefore access it using the index 0. We can use this with string slicing to grab everything from the first charchter in our string to the 18th last character(The length of the string ' - Rotten Tomatoes'). 

In [9]:
soup.find('title').contents[0][:-len(' - Rotten Tomatoes')]

'E.T. The Extra-Terrestrial\xa0(1982)'

The Jupyter Notebook below contains template code that:

* Creates an empty list, df_list, to which dictionaries will be appended. This list of dictionaries will eventually be converted to a pandas DataFrame (this is the most efficient way of building a DataFrame row by row).
* Loops through each movie's Rotten Tomatoes HTML file in the rt_html folder.
* Opens each HTML file and passes it into a file handle called file.
* Creates a DataFrame called df by converting df_list using the pd.DataFrame constructor.

Your task is to extract the title, audience score, and number of audience ratings in each HTML file so each trio can be appended as a dictionary to df_list.

In [10]:
import os

# List of dictionaries to build file by file and later convert to a DataFrame
df_list = []
folder = 'rt_html'
for movie_html in os.listdir(folder):
    with open(os.path.join(folder, movie_html)) as file:
        # Make the soup
        soup = BeautifulSoup(file, "html.parser")
        
        #Find title. We don't wnat the '- Rotten Tomatoes' part of the title so only take the part before that
        title = soup.find('title').contents[0][:-len(' - Rotten tomatoes')]
        
        #First find the div with class name audience-score meter then find the only span element within and grab the 
        # contents without the % sign
        audience_score = soup.find('div', class_='audience-score meter').find('span').contents[0][:-1]
        
        #Find the div with number of ratings
        num_audience_ratings = soup.find('div', class_ = 'audience-info hidden-xs superPageFontColor')
        
        #Find the second span tag within the div which contains number of reviews just want number of reviews and 
        # strip out whitespace. Will need to convert string to int so have to replace comma with empty
        num_audience_ratings = num_audience_ratings.find_all('div')[1].contents[2].strip().replace(',', '')

## Roger Ebert review
We are going to look at Rogert Eberts reviews for each of the movies listed in Rotten Tomatoes top 100 movies. The files for these reviews have been stored on a Udacity page. We will download them programatically using Pyhtons **Request** libraray.

Python requests library has a method called **.get** which will send the request for us, return the contents of the file we requested, which we can then save to a file. 

Here's how it's done:

In [11]:
import requests
import os

folder_name = 'ebert_review'
if not os.path.exists(folder_name): #create folder if it doesn't exist already
    os.makedirs(folder_name)
    
# Request code:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9904_11-e.t.-the-extra-terrestrial/11-e.t.-the-extra-terrestrial.txt'
response=requests.get(url)
# We haven't actually saved the response to anything yet but let's take a look at what the response looks like
response

<Response [200]>

Response code 200 means everything went well with the request.

All the text in our text file is in our computers working memory right now within this response variable. It's stored in the body of the response which we can access using **.content** 

In [12]:
response.content

b'E.T. The Extra-Terrestrial (1982)\nhttp://www.rogerebert.com/reviews/great-movie-et-the-extra-terrestrial-1982\nDear Raven and Emil:\n\nSunday we sat on the big green couch and watched "E.T. The Extra-Terrestrial" together with your mommy and daddy. It was the first time either of you had seen it, although you knew a little of what to expect because we took the "E.T." ride together at the Universal tour. I had seen the movie lots of times since it came out in 1982, so I kept one eye on the screen and the other on the two of you. I wanted to see how a boy on his fourth birthday, and a girl who had just turned 7 a week ago, would respond to the movie.\n\nWell, it "worked" for both of you, as we say in Grandpa Roger\'s business.\n\nRaven, you never took your eyes off the screen--not even when it looked like E.T. was dying and you had to scoot over next to me because you were afraid.\n\nEmil, you had to go sit on your dad\'s knee a couple of times, but you never stopped watching, either.

* Response is in in Bytes format. 
* Using this and some basic file IO we're going to save this file to our computer. 
* We'll open a file called '11-e.t.-the-extra-terrestrial.txt', i.e. everything after the last / in the url. In order to get everything after the last / we'll use Python's split function. 
* We need to open the contents of this file which we'll then write the contents of the response variable to. We have to open this is wb mode (write binary).

In [13]:
with open(os.path.join(folder_name,
                      url.split('/')[-1]), mode='wb') as file:
    file.write(response.content)

That's how you download 1 file programatically. Let's check the contents of our folder ebert_reviews to make sure it worked:

In [14]:
os.listdir(folder_name)

['26-dunkirk-2017-film.txt',
 '57-army-of-shadows.txt',
 '48-alien-film.txt',
 '43-bride-of-frankenstein.txt',
 '54-the-400-blows.txt',
 '63-manchester-by-the-sea-film.txt',
 '5-mad-max-fury-road.txt',
 '46-wonder-woman-2017-film.txt',
 '49-bicycle-thieves.txt',
 '19-laura-1944-film.txt',
 '98-toy-story.txt',
 '90-on-the-waterfront.txt',
 '39-toy-story-3.txt',
 '82-tokyo-story.txt',
 '23-la-grande-illusion.txt',
 '92-the-last-picture-show.txt',
 '36-rear-window.txt',
 '78-gone-with-the-wind-film.txt',
 '60-a-streetcar-named-desire-1951-film.txt',
 '84-pinocchio-1940-film.txt',
 '32-king-kong-1933-film.txt',
 '58-arrival-film.txt',
 '64-dr.-strangelove.txt',
 '40-argo-2012-film.txt',
 '22-a-hard-day27s-night-film.txt',
 '37-selma-film.txt',
 '83-hell-or-high-water-film.txt',
 '30-gravity-2013-film.txt',
 '55-logan-film.txt',
 '10-metropolis-1927-film.txt',
 '42-the-big-sick.txt',
 '72-rosemary27s-baby-film.txt',
 '53-12-angry-men-1957-film.txt',
 '4-get-out-film.txt',
 '11-e.t.-the-extr

Since we have 100 files to download we'll use a for loop in conjunction with the provided ebert_review_urls list.

In [15]:
ebert_review_urls = ['https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9900_1-the-wizard-of-oz-1939-film/1-the-wizard-of-oz-1939-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9901_2-citizen-kane/2-citizen-kane.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9901_3-the-third-man/3-the-third-man.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9902_4-get-out-film/4-get-out-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9902_5-mad-max-fury-road/5-mad-max-fury-road.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9902_6-the-cabinet-of-dr.-caligari/6-the-cabinet-of-dr.-caligari.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9903_7-all-about-eve/7-all-about-eve.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9903_8-inside-out-2015-film/8-inside-out-2015-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9903_9-the-godfather/9-the-godfather.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9904_10-metropolis-1927-film/10-metropolis-1927-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9904_11-e.t.-the-extra-terrestrial/11-e.t.-the-extra-terrestrial.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9904_12-modern-times-film/12-modern-times-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9904_14-singin-in-the-rain/14-singin-in-the-rain.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9905_15-boyhood-film/15-boyhood-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9905_16-casablanca-film/16-casablanca-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9905_17-moonlight-2016-film/17-moonlight-2016-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9906_18-psycho-1960-film/18-psycho-1960-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9906_19-laura-1944-film/19-laura-1944-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9906_20-nosferatu/20-nosferatu.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9907_21-snow-white-and-the-seven-dwarfs-1937-film/21-snow-white-and-the-seven-dwarfs-1937-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9907_22-a-hard-day27s-night-film/22-a-hard-day27s-night-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9907_23-la-grande-illusion/23-la-grande-illusion.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9908_25-the-battle-of-algiers/25-the-battle-of-algiers.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9908_26-dunkirk-2017-film/26-dunkirk-2017-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9908_27-the-maltese-falcon-1941-film/27-the-maltese-falcon-1941-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9909_29-12-years-a-slave-film/29-12-years-a-slave-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9909_30-gravity-2013-film/30-gravity-2013-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9909_31-sunset-boulevard-film/31-sunset-boulevard-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad990a_32-king-kong-1933-film/32-king-kong-1933-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad990a_33-spotlight-film/33-spotlight-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad990a_34-the-adventures-of-robin-hood/34-the-adventures-of-robin-hood.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad990b_35-rashomon/35-rashomon.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad990b_36-rear-window/36-rear-window.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad990b_37-selma-film/37-selma-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad990c_38-taxi-driver/38-taxi-driver.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad990c_39-toy-story-3/39-toy-story-3.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad990c_40-argo-2012-film/40-argo-2012-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad990d_41-toy-story-2/41-toy-story-2.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad990d_42-the-big-sick/42-the-big-sick.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad990d_43-bride-of-frankenstein/43-bride-of-frankenstein.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad990d_44-zootopia/44-zootopia.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad990e_45-m-1931-film/45-m-1931-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad990e_46-wonder-woman-2017-film/46-wonder-woman-2017-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad990e_48-alien-film/48-alien-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad990f_49-bicycle-thieves/49-bicycle-thieves.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad990f_50-seven-samurai/50-seven-samurai.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad990f_51-the-treasure-of-the-sierra-madre-film/51-the-treasure-of-the-sierra-madre-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9910_52-up-2009-film/52-up-2009-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9910_53-12-angry-men-1957-film/53-12-angry-men-1957-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9910_54-the-400-blows/54-the-400-blows.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9911_55-logan-film/55-logan-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9911_57-army-of-shadows/57-army-of-shadows.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9912_58-arrival-film/58-arrival-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9912_59-baby-driver/59-baby-driver.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9913_60-a-streetcar-named-desire-1951-film/60-a-streetcar-named-desire-1951-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9913_61-the-night-of-the-hunter-film/61-the-night-of-the-hunter-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9913_62-star-wars-the-force-awakens/62-star-wars-the-force-awakens.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9913_63-manchester-by-the-sea-film/63-manchester-by-the-sea-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9914_64-dr.-strangelove/64-dr.-strangelove.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9914_66-vertigo-film/66-vertigo-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9914_67-the-dark-knight-film/67-the-dark-knight-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9915_68-touch-of-evil/68-touch-of-evil.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9915_69-the-babadook/69-the-babadook.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9915_72-rosemary27s-baby-film/72-rosemary27s-baby-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9916_73-finding-nemo/73-finding-nemo.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9916_74-brooklyn-film/74-brooklyn-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9917_75-the-wrestler-2008-film/75-the-wrestler-2008-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9917_77-l.a.-confidential-film/77-l.a.-confidential-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9918_78-gone-with-the-wind-film/78-gone-with-the-wind-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9918_79-the-good-the-bad-and-the-ugly/79-the-good-the-bad-and-the-ugly.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9918_80-skyfall/80-skyfall.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9919_82-tokyo-story/82-tokyo-story.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9919_83-hell-or-high-water-film/83-hell-or-high-water-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9919_84-pinocchio-1940-film/84-pinocchio-1940-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9919_85-the-jungle-book-2016-film/85-the-jungle-book-2016-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad991a_86-la-la-land-film/86-la-la-land-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad991b_87-star-trek-film/87-star-trek-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad991b_89-apocalypse-now/89-apocalypse-now.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad991c_90-on-the-waterfront/90-on-the-waterfront.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad991c_91-the-wages-of-fear/91-the-wages-of-fear.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad991c_92-the-last-picture-show/92-the-last-picture-show.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad991d_93-harry-potter-and-the-deathly-hallows-part-2/93-harry-potter-and-the-deathly-hallows-part-2.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad991d_94-the-grapes-of-wrath-film/94-the-grapes-of-wrath-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad991d_96-man-on-wire/96-man-on-wire.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad991e_97-jaws-film/97-jaws-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad991e_98-toy-story/98-toy-story.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad991e_99-the-godfather-part-ii/99-the-godfather-part-ii.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad991e_100-battleship-potemkin/100-battleship-potemkin.txt']

In [16]:
# Our text is stored in the body of the response which we can get using .content
# Text stored in bytes format so open in mode='wb' 
for url in ebert_review_urls:
# Use GET to obtain data from URL
    response = requests.get(url)
    with open(os.path.join(folder_name,
# For file name get everything after the last slash in the url
# Use split fxn to get last item in the list returned
                      url.split('/')[-1]), mode='wb') as file:
# Write to file handle we've opened
        file.write(response.content)

In [17]:
# Check to make sure the files were downloaded
os.listdir(folder_name)

['26-dunkirk-2017-film.txt',
 '57-army-of-shadows.txt',
 '48-alien-film.txt',
 '43-bride-of-frankenstein.txt',
 '54-the-400-blows.txt',
 '63-manchester-by-the-sea-film.txt',
 '5-mad-max-fury-road.txt',
 '46-wonder-woman-2017-film.txt',
 '49-bicycle-thieves.txt',
 '19-laura-1944-film.txt',
 '98-toy-story.txt',
 '90-on-the-waterfront.txt',
 '39-toy-story-3.txt',
 '82-tokyo-story.txt',
 '23-la-grande-illusion.txt',
 '92-the-last-picture-show.txt',
 '36-rear-window.txt',
 '78-gone-with-the-wind-film.txt',
 '60-a-streetcar-named-desire-1951-film.txt',
 '84-pinocchio-1940-film.txt',
 '32-king-kong-1933-film.txt',
 '58-arrival-film.txt',
 '64-dr.-strangelove.txt',
 '40-argo-2012-film.txt',
 '22-a-hard-day27s-night-film.txt',
 '37-selma-film.txt',
 '83-hell-or-high-water-film.txt',
 '30-gravity-2013-film.txt',
 '55-logan-film.txt',
 '10-metropolis-1927-film.txt',
 '42-the-big-sick.txt',
 '72-rosemary27s-baby-film.txt',
 '53-12-angry-men-1957-film.txt',
 '4-get-out-film.txt',
 '11-e.t.-the-extr

## Extracting text from Ebert review

Need to select the right encoding to display the character sets properly. 

### Unicode and Python
In Python 3, there is:
* one text type: str, which holds Unicode data and
* two byte types: bytes and bytearray
    

Gathering data from text files in Python means opening and reading from files. If you're using Pandas then this also means storing the text data you just read in a Pandas dataframe. 

We have 88 review to open and read. We'll need a loop to iterate through all of the files in the folder to open and read each. There are 2 main ways of doing this:
1. Using the os library
2. Using a libray called **glob**

We've been using os.listdir so far which is good if you're sure you wnt to open every file in the folder. Let's switch it up and use glob instead. The glob library allows for using glob patterns to specify sets of file names. These glob patterns use wildcard characters. 

* **glob.glob** returns a list of path names that match *pathname*, i.e. the string aprameter you pass in
We can use this to find all files that end in .txt, which in our folder is all of them. This returns a list that we can then loop through.
* Make sure to **include encoding when opeing a file**. In order to find out what encoding to use we can go toone of the review pages on Robert Eberts site and **inspect source.** 
* When we open the file we don't want all of the text which would be done using print(file.read()). We want the following as seperate pieces of data:
    * the first line a.k.a. the movie title (to merge to the master dataset with)
    * the second line a.k.a. the review URL (not necessary for the word cloud but nice to have)
    * everything from the third line onwards a.k.a. the review text
* Since text files are seperated by new line characters and the file object returned from *with open as file* is an iterator we can read the file line by line. If you just want to read one line you use **.readline()**
* There's actually a bit of whitespace after the movie titele which is the newline charcater. We can get rid of that by slicing it off at the end of the title


In [18]:
import glob

for ebert_review in glob.glob('ebert_review/*.txt'):
    with open(ebert_review) as file: 
        title = file.readline()[:-1]       

We also want the review URL and the view text. But first we want all of this **data in a Pandas dataframe** so we need to build one. The most computationally efficient way to do that is to **create an empty list** and populate that list one by one as we iterate through this loop. We'll **fill this list with dictionaries** and the list of dictionaries will be later converted to a pnadas dataframe.  

In [19]:
# List of dictionaries to build file by file and later convert to a DataFrame
df_list = []
for ebert_review in glob.glob('ebert_reviews/*.txt'):
    with open(ebert_review) as file:
        title = file.readline()[:-1]
        review_url = file.readline()[:-1]
        review_text = file.read()

        # Append to list of dictionaries
        df_list.append({'title': title,
                        'review_url': review_url,
                        'review_text': review_text})
df = pd.DataFrame(df_list, columns = ['title', 'review_url', 'review_text'])

In [20]:
df.head()

,title,review_url,review_text
0,Dunkirk (2017),http://www.rogerebert.com/reviews/dunkirk-2017,"Lean and ambitious, unsentimental and bombasti..."
1,Army of Shadows (L'Armée des ombres) (1969),http://www.rogerebert.com/reviews/great-movie-...,"Jean-Pierre Melville's ""Army of Shadows"" is ab..."
2,Alien (1979),http://www.rogerebert.com/reviews/great-movie-...,"At its most fundamental level, ""Alien"" is a mo..."
3,The Bride of Frankenstein (1935),http://www.rogerebert.com/reviews/great-movie-...,To a new world of gods and monsters.\n\nSo int...
4,The 400 Blows (Les Quatre cents coups) (1959),http://www.rogerebert.com/reviews/great-movie-...,I demand that a film express either the joy of...


## Source: APIs 

Now we're going to get each movies poster. Since each movie has it's poster on it's Wikipedia page we can use Wikipedia's API to get the images. We'll be using the **Mediawiki API**.

We could have used Rotten Tomatoes API and the **rtsimple** libraray to get the Audience and critic scores. here's how it would have worked:

In [21]:
# Note:need API key for code to run
#import rtsimple as rt

#rt.API_KEY = "YOUR KEY HERE"
#create movie object with the movie ID for E.T.
#movie = rt.Movies('10489')
#Access movie ratings
#movie.ratings['audience_score']

Because we can't use the Rotten Tomatoes API (requires proposal form) we're going to use MediaWiki along with **wptools** library.

* To get a **page object**, the usage is as follows: *page = wptools.page('Mahatma_Gandhi')* where 'Mahatma_Gandhi' is the last bit of the Wikipedia URL for that page. This page object has methods that can get us various pieces of data about that Wikipedia page, including all of the images on the page. 
* To get all of the data: *page = wptools.page('Mahatma_Gandhi').get()*
* page now has the all associated attributes (such as image, infobox, links etc.), which can be accessed using dot notation through .data. e.g. *page.data['image']* would return a list of data for six images on this specific Wikipedia page.

Now we'll get the page object for the E.T. The Extra-Terrestial Wikipedia page.

In [22]:
import wptools

page = wptools.page('E.T._the_Extra-Terrestrial').get()

# Accessing the image attribute will return the images for this page
page.data['image']

en.wikipedia.org (query) E.T._the_Extra-Terrestrial
en.wikipedia.org (parse) 73441
www.wikidata.org (wikidata) Q11621
www.wikidata.org (labels) P18|P3593|P58|P3145|Q3897561|P3212|Q488...
www.wikidata.org (labels) Q1860|P1712|P2529|Q3953565|Q652644|Q686...
www.wikidata.org (labels) P905|P921|P3844|P345|P57|P2631|P166|Q28...
www.wikidata.org (labels) Q471839|Q952914|Q1044183|Q918617|P3203|P1237
en.wikipedia.org (restbase) /page/summary/E.T._the_Extra-Terrestrial
en.wikipedia.org (imageinfo) File:E t the extra terrestrial ver3....
E.T. the Extra-Terrestrial (en) data
{
  aliases: <list(2)> E.T., ET
  claims: <dict(78)> P86, P18, P3593, P58, P268, P3145, P244, P321...
  description: 1982 American science fiction film
  exhtml: <str(825)> <p><i><b>E.T. the Extra-Terrestrial</b></i> i...
  exrest: <str(794)> E.T. the Extra-Terrestrial is a 1982 American...
  extext: <str(1808)> _**E.T. the Extra-Terrestrial**_ is a 1982 A...
  extract: <str(1898)> <p><i><b>E.T. the Extra-Terrestrial</b></i>.

[{'descriptionshorturl': 'https://en.wikipedia.org/w/index.php?curid=7419503',
  'descriptionurl': 'https://en.wikipedia.org/wiki/File:E_t_the_extra_terrestrial_ver3.jpg',
  'file': 'File:E t the extra terrestrial ver3.jpg',
  'height': 394,
  'kind': 'parse-image',
  'orig': 'E t the extra terrestrial ver3.jpg',
  'size': 83073,
  'timestamp': '2016-06-04T10:30:46Z',
  'title': 'File:E t the extra terrestrial ver3.jpg',
  'url': 'https://upload.wikimedia.org/wikipedia/en/6/66/E_t_the_extra_terrestrial_ver3.jpg',
  'width': 253},
 {'descriptionshorturl': 'https://commons.wikimedia.org/w/index.php?curid=14138952',
  'descriptionurl': 'https://commons.wikimedia.org/wiki/File:ET_logo_3.svg',
  'file': 'File:ET logo 3.svg',
  'height': 380,
  'kind': 'wikidata-image',
  'orig': 'ET logo 3.svg',
  'size': 77290,
  'timestamp': '2011-05-14T20:38:17Z',
  'title': 'File:ET logo 3.svg',
  'url': 'https://upload.wikimedia.org/wikipedia/commons/8/85/ET_logo_3.svg',
  'width': 512},
 {'file': 'Fil

## JSON Files in Python

Most data from APIs come in JSON or XML format. We'll focus on JSON.

Sometimes we are limited in what we can represent in tabular data. 
* We might have fields that have multiple entries, like producers for a movie. 
* Fields may have subfields, like release date having a date and location of release.
JSON is great for representing and accessing complicated data heirachies, like our Wikipedia infobox.

* **JSON objects** are a collection of key value pairs. Objects are surrounded by curly braces. In Python JSON objects are **interpreted as dictionaries** and you can access them in the same way. 
* **JSON arrays** are an ordered list of values denoted by square brackets. In Python, JSON arrays are **interpreted as lists** and are accessed in the same way. 


Let's inspect the wptools page object for the E.T. The Extra-Terrestial Wikipedia page. 

In [23]:

page = wptools.page('E.T._the_Extra-Terrestrial').get()

en.wikipedia.org (query) E.T._the_Extra-Terrestrial
en.wikipedia.org (parse) 73441
www.wikidata.org (wikidata) Q11621
www.wikidata.org (labels) P18|P3593|P58|P3145|Q3897561|P3212|Q488...
www.wikidata.org (labels) Q1860|P1712|P2529|Q3953565|Q652644|Q686...
www.wikidata.org (labels) P905|P921|P3844|P345|P57|P2631|P166|Q28...
www.wikidata.org (labels) Q471839|Q952914|Q1044183|Q918617|P3203|P1237
en.wikipedia.org (restbase) /page/summary/E.T._the_Extra-Terrestrial
en.wikipedia.org (imageinfo) File:E t the extra terrestrial ver3....
E.T. the Extra-Terrestrial (en) data
{
  aliases: <list(2)> E.T., ET
  claims: <dict(78)> P86, P18, P3593, P58, P268, P3145, P244, P321...
  description: 1982 American science fiction film
  exhtml: <str(825)> <p><i><b>E.T. the Extra-Terrestrial</b></i> i...
  exrest: <str(794)> E.T. the Extra-Terrestrial is a 1982 American...
  extext: <str(1808)> _**E.T. the Extra-Terrestrial**_ is a 1982 A...
  extract: <str(1898)> <p><i><b>E.T. the Extra-Terrestrial</b></i>.

Access the first image in the images attribute, which is a JSON array.

In [24]:
page.data['image'][0]

{'descriptionshorturl': 'https://en.wikipedia.org/w/index.php?curid=7419503',
 'descriptionurl': 'https://en.wikipedia.org/wiki/File:E_t_the_extra_terrestrial_ver3.jpg',
 'file': 'File:E t the extra terrestrial ver3.jpg',
 'height': 394,
 'kind': 'parse-image',
 'orig': 'E t the extra terrestrial ver3.jpg',
 'size': 83073,
 'timestamp': '2016-06-04T10:30:46Z',
 'title': 'File:E t the extra terrestrial ver3.jpg',
 'url': 'https://upload.wikimedia.org/wikipedia/en/6/66/E_t_the_extra_terrestrial_ver3.jpg',
 'width': 253}

Access the director key of the infobox attribute, which is a JSON object.

In [25]:
page.data['infobox']['director']

'[[Steven Spielberg]]'

## APIs, Downloading Files Programmatically, and JSON

There are two key things to be aware of before you begin:
1. Wikipedia Page Titles. To access Wikipedia page data via the MediaWiki API with wptools, you need each movie's Wikipedia page title, i.e., what comes after the last slash in en.wikipedia.org/wiki/ in the URL. For this lesson,  the titles for each of the movies in the Top 100 has been complied.
2. Downloading Image Files. You can use regular file opening, reading, and writing techniques, like this:

In [26]:
#import requests
#r = requests.get(url)
#with open(folder_name + '/' + filename, 'wb') as f:
#        f.write(r.content)

But this technique can be error-prone. It will work most of the time, but sometimes the file you write to will be damaged. This type of error is why the requests library maintainers recommend using the PIL library (short for Pillow) and BytesIO from the io library for non-text requests, like images. They recommend that you access the response body as bytes, for non-text requests. For example, to create an image from binary data returned by a request:

In [27]:
#import requests
#from PIL import Image
#from io import BytesIO
#r = requests.get(url)
#i = Image.open(BytesIO(r.content))

Though you may still encounter a similar file error, this code above will at least warn us with an error message, at which point we can manually download the problematic images.

Let's gather the last piece of data for the Roger Ebert review word clouds now: the movie poster image files. Let's also keep each image's URL to add to the master DataFrame later.

Though we're going to use a loop to minimize repetition, here's how the major parts inside that loop will work, in order:

1. We're going to query the MediaWiki API using wptools to get a movie poster URL via each page object's image attribute.
2. Using that URL, we'll programmatically download that image into a folder called bestofrt_posters.

The code below contains code that:

* Contains title_list, which is a list of all of the Wikipedia page titles for each movie in the Rotten Tomatoes Top 100 Movies of All Time list. This list is in the same order as the Top 100.
* Creates an empty list, df_list, to which dictionaries will be appended. This list of dictionaries will eventually be converted to a pandas DataFrame (this is the most efficient way of building a DataFrame row by row).
* Creates an empty folder, bestofrt_posters, to store the downloaded movie poster image files.
* Creates an empty dictionary, image_errors, to fill to keep track of movie poster image URLs that don't work.
* Loops through the Wikipedia page titles in title_list and:
    * Stores the ranking of that movie in the Top 100 list based on its position in title_list. Ranking is needed so we can join this DataFrame with the master DataFrame later. We can't join on title because the titles of the Rotten Tomatoes pages and the Wikipedia pages differ.
    * Uses try and except blocks to attempt to query MediaWiki for a movie poster image URL and to attempt to download that image. If the attempt fails and an error is encountered, the offending movie is documented in image_errors.
    * Appends a dictionary with ranking, title, and poster_url as the keys and the extracted values for each as the values to df_list.
* Inspects the images that caused errors and downloads the correct image individually (either via another URL in the image attribute's list or a URL from Google Images)
* Creates a DataFrame called df by converting df_list using the pd.DataFrame constructor.

In [29]:
from PIL import Image
from io import BytesIO

title_list = [
 'The_Wizard_of_Oz_(1939_film)',
 'Citizen_Kane',
 'The_Third_Man',
 'Get_Out_(film)',
 'Mad_Max:_Fury_Road',
 'The_Cabinet_of_Dr._Caligari',
 'All_About_Eve',
 'Inside_Out_(2015_film)',
 'The_Godfather',
 'Metropolis_(1927_film)',
 'E.T._the_Extra-Terrestrial',
 'Modern_Times_(film)',
 'It_Happened_One_Night',
 "Singin'_in_the_Rain",
 'Boyhood_(film)',
 'Casablanca_(film)',
 'Moonlight_(2016_film)',
 'Psycho_(1960_film)',
 'Laura_(1944_film)',
 'Nosferatu',
 'Snow_White_and_the_Seven_Dwarfs_(1937_film)',
 "A_Hard_Day%27s_Night_(film)",
 'La_Grande_Illusion',
 'North_by_Northwest',
 'The_Battle_of_Algiers',
 'Dunkirk_(2017_film)',
 'The_Maltese_Falcon_(1941_film)',
 'Repulsion_(film)',
 '12_Years_a_Slave_(film)',
 'Gravity_(2013_film)',
 'Sunset_Boulevard_(film)',
 'King_Kong_(1933_film)',
 'Spotlight_(film)',
 'The_Adventures_of_Robin_Hood',
 'Rashomon',
 'Rear_Window',
 'Selma_(film)',
 'Taxi_Driver',
 'Toy_Story_3',
 'Argo_(2012_film)',
 'Toy_Story_2',
 'The_Big_Sick',
 'Bride_of_Frankenstein',
 'Zootopia',
 'M_(1931_film)',
 'Wonder_Woman_(2017_film)',
 'The_Philadelphia_Story_(film)',
 'Alien_(film)',
 'Bicycle_Thieves',
 'Seven_Samurai',
 'The_Treasure_of_the_Sierra_Madre_(film)',
 'Up_(2009_film)',
 '12_Angry_Men_(1957_film)',
 'The_400_Blows',
 'Logan_(film)',
 'All_Quiet_on_the_Western_Front_(1930_film)',
 'Army_of_Shadows',
 'Arrival_(film)',
 'Baby_Driver',
 'A_Streetcar_Named_Desire_(1951_film)',
 'The_Night_of_the_Hunter_(film)',
 'Star_Wars:_The_Force_Awakens',
 'Manchester_by_the_Sea_(film)',
 'Dr._Strangelove',
 'Frankenstein_(1931_film)',
 'Vertigo_(film)',
 'The_Dark_Knight_(film)',
 'Touch_of_Evil',
 'The_Babadook',
 'The_Conformist_(film)',
 'Rebecca_(1940_film)',
 "Rosemary%27s_Baby_(film)",
 'Finding_Nemo',
 'Brooklyn_(film)',
 'The_Wrestler_(2008_film)',
 'The_39_Steps_(1935_film)',
 'L.A._Confidential_(film)',
 'Gone_with_the_Wind_(film)',
 'The_Good,_the_Bad_and_the_Ugly',
 'Skyfall',
 'Rome,_Open_City',
 'Tokyo_Story',
 'Hell_or_High_Water_(film)',
 'Pinocchio_(1940_film)',
 'The_Jungle_Book_(2016_film)',
 'La_La_Land_(film)',
 'Star_Trek_(film)',
 'High_Noon',
 'Apocalypse_Now',
 'On_the_Waterfront',
 'The_Wages_of_Fear',
 'The_Last_Picture_Show',
 'Harry_Potter_and_the_Deathly_Hallows_–_Part_2',
 'The_Grapes_of_Wrath_(film)',
 'Roman_Holiday',
 'Man_on_Wire',
 'Jaws_(film)',
 'Toy_Story',
 'The_Godfather_Part_II',
 'Battleship_Potemkin'
]

folder_name = 'bestofrt_posters'
# Make directory if it doesn't already exist
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    
# List of dictionaries to build and convert to a DataFrame later
df_list = []
image_errors = {}
for title in title_list:
    try:
        # This cell is slow so print ranking to gauge time remaining
        ranking = title_list.index(title) + 1
        print(ranking)
        page = wptools.page(title, silent=True)
        # Your code here (three lines)
        images = page.get().data['image']
        # First image is usually the poster
        first_image_url = images[0]['url']
        r = requests.get(first_image_url)
        # Download movie poster image
        i = Image.open(BytesIO(r.content))
        image_file_format = first_image_url.split('.')[-1]
        i.save(folder_name + "/" + str(ranking) + "_" + title + '.' + image_file_format)
        # Append to list of dictionaries
        df_list.append({'ranking': int(ranking),
                        'title': title,
                        'poster_url': first_image_url})
    
    # Not best practice to catch all exceptions but fine for this short script
    except Exception as e:
        print(str(ranking) + "_" + title + ": " + str(e))
        image_errors[str(ranking) + "_" + title] = images
        


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22


API error: {'info': 'Bad title "A_Hard_Day%27s_Night_(film)".', 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.', 'code': 'invalidtitle'}


22_A_Hard_Day%27s_Night_(film): https://en.wikipedia.org/w/api.php?action=parse&formatversion=2&contentmodel=text&disableeditsection=&disablelimitreport=&disabletoc=&prop=text|iwlinks|parsetree|wikitext|displaytitle|properties&redirects&page=A_Hard_Day%2527s_Night_%28film%29
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
53_12_Angry_Men_(1957_film): cannot identify image file <_io.BytesIO object at 0x1186464c8>
54
55
56
57
58
59
60
61
62
63
64
64_Dr._Strangelove: cannot identify image file <_io.BytesIO object at 0x11865db28>
65
66
67
68
69
70
71
72


API error: {'info': 'Bad title "Rosemary%27s_Baby_(film)".', 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.', 'code': 'invalidtitle'}


72_Rosemary%27s_Baby_(film): https://en.wikipedia.org/w/api.php?action=parse&formatversion=2&contentmodel=text&disableeditsection=&disablelimitreport=&disabletoc=&prop=text|iwlinks|parsetree|wikitext|displaytitle|properties&redirects&page=Rosemary%2527s_Baby_%28film%29
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
93_Harry_Potter_and_the_Deathly_Hallows_–_Part_2: cannot identify image file <_io.BytesIO object at 0x1183f3108>
94
95
96
97
98
99
100


One you have completed the above code requirements, read and run the three cells below and interpret their output.

In [30]:
for key in image_errors.keys():
    print(key)

22_A_Hard_Day%27s_Night_(film)
93_Harry_Potter_and_the_Deathly_Hallows_–_Part_2
64_Dr._Strangelove
72_Rosemary%27s_Baby_(film)
53_12_Angry_Men_(1957_film)


In [31]:
# Inspect unidentifiable images and download them individually
for rank_title, images in image_errors.items():
    if rank_title == '22_A_Hard_Day%27s_Night_(film)':
        url = 'https://upload.wikimedia.org/wikipedia/en/4/47/A_Hard_Days_night_movieposter.jpg'
    if rank_title == '53_12_Angry_Men_(1957_film)':
        url = 'https://upload.wikimedia.org/wikipedia/en/9/91/12_angry_men.jpg'
    if rank_title == '72_Rosemary%27s_Baby_(film)':
        url = 'https://upload.wikimedia.org/wikipedia/en/e/ef/Rosemarys_baby_poster.jpg'
    if rank_title == '93_Harry_Potter_and_the_Deathly_Hallows_–_Part_2':
        url = 'https://upload.wikimedia.org/wikipedia/en/d/df/Harry_Potter_and_the_Deathly_Hallows_%E2%80%93_Part_2.jpg'
    title = rank_title[3:]
    df_list.append({'ranking': int(title_list.index(title) + 1),
                    'title': title,
                    'poster_url': url})
    r = requests.get(url)
    # Download movie poster image
    i = Image.open(BytesIO(r.content))
    image_file_format = url.split('.')[-1]
    i.save(folder_name + "/" + rank_title + '.' + image_file_format)

In [32]:
# Create DataFrame from list of dictionaries
df = pd.DataFrame(df_list, columns = ['ranking', 'title', 'poster_url'])
df = df.sort_values('ranking').reset_index(drop=True)
df

,ranking,title,poster_url
0,1,The_Wizard_of_Oz_(1939_film),https://upload.wikimedia.org/wikipedia/commons...
1,2,Citizen_Kane,https://upload.wikimedia.org/wikipedia/en/c/ce...
2,3,The_Third_Man,https://upload.wikimedia.org/wikipedia/en/2/21...
3,4,Get_Out_(film),https://upload.wikimedia.org/wikipedia/en/e/eb...
4,5,Mad_Max:_Fury_Road,https://upload.wikimedia.org/wikipedia/en/6/6e...
5,6,The_Cabinet_of_Dr._Caligari,https://upload.wikimedia.org/wikipedia/commons...
6,7,All_About_Eve,https://upload.wikimedia.org/wikipedia/en/2/22...
7,8,Inside_Out_(2015_film),https://upload.wikimedia.org/wikipedia/en/0/0a...
8,9,The_Godfather,https://upload.wikimedia.org/wikipedia/en/1/1c...
9,10,Metropolis_(1927_film),https://upload.wikimedia.org/wikipedia/en/0/06...


## Storing data

Saving to a csv file is best for a simple datatframe like this

In [33]:
# We'll take some data that's already been cleaned since our data here hasn't been cleaned yet
df = pd.read_csv('gathered_assessed_cleaned.csv')

# Save the master DataFrame to a file called 'bestofrt_master.csv'
df.to_csv('bestofrt_master.csv', index=False)

## Relational databases in Python

We're going to:

1. Connect to a database. We'll connect to a SQLite database using SQLAlchemy, a database toolkit for Python.
2. Store the data in the cleaned master dataset in that database. We'll do this using pandas' to_csv DataFrame method.
3. Then read the brand new data in that database back into a pandas DataFrame. We'll do this using pandas' read_csv function.

In [34]:
df = pd.read_csv('bestofrt_master.csv')
df.head()

,ranking,title,critic_score,number_of_critic_ratings,audience_score,number_of_audience_ratings,review_url,review_text,poster_url
0,1,The Wizard of Oz (1939),99,110,89,874425,http://www.rogerebert.com/reviews/great-movie-...,As a child I simply did not notice whether a m...,https://upload.wikimedia.org/wikipedia/commons...
1,2,Citizen Kane (1941),100,75,90,157274,http://www.rogerebert.com/reviews/great-movie-...,“I don't think any word can explain a man's li...,https://upload.wikimedia.org/wikipedia/en/c/ce...
2,3,The Third Man (1949),100,77,93,53081,http://www.rogerebert.com/reviews/great-movie-...,Has there ever been a film where the music mor...,https://upload.wikimedia.org/wikipedia/en/2/21...
3,4,Get Out (2017),99,282,87,63837,http://www.rogerebert.com/reviews/get-out-2017,"With the ambitious and challenging “Get Out,” ...",https://upload.wikimedia.org/wikipedia/en/e/eb...
4,5,Mad Max: Fury Road (2015),97,370,86,123937,http://www.rogerebert.com/reviews/mad-max-fury...,George Miller’s “Mad Max” films didn’t just ma...,https://upload.wikimedia.org/wikipedia/en/6/6e...


### 1. Connect to a database

In [35]:
from sqlalchemy import create_engine

# Create SQLAlchemy Engine and empty bestofrt database
# bestofrt.db will not show up in the Jupyter Notebook dashboard yet
engine = create_engine('sqlite:///bestofrt.db')

ImportError: No module named 'sqlalchemy'

### 2. Store pandas DataFrame in database
Store the data in the cleaned master dataset (bestofrt_master) in that database.

In [ ]:
# Store cleaned master DataFrame ('df') in a table called master in bestofrt.db
# bestofrt.db will be visible now in the Jupyter Notebook dashboard
df.to_sql('master', engine, index=False)

### 3. Read database data into a pandas DataFrame
Read the brand new data in that database back into a pandas DataFrame.

In [ ]:
df_gather = pd.read_sql('SELECT * FROM master', engine)
df_gather.head(3)